In [9]:
from abc import ABCMeta,abstractmethod


class Clasificador:

    # Clase abstracta
    __metaclass__ = ABCMeta

    # Metodos abstractos que se implementan en casa clasificador concreto
    @abstractmethod
    # TODO: esta funcion debe ser implementada en cada clasificador concreto
    # datosTrain: matriz numpy con los datos de entrenamiento
    # atributosDiscretos: array bool con la indicatriz de los atributos nominales
    # diccionario: array de diccionarios de la estructura Datos utilizados para la codificacion de variables discretas
    def entrenamiento(self,datosTrain,atributosDiscretos,diccionario):
        pass


    @abstractmethod
    # TODO: esta funcion debe ser implementada en cada clasificador concreto
    # devuelve un numpy array con las predicciones
    def clasifica(self,datosTest,atributosDiscretos,diccionario):
        pass


    # Obtiene el numero de aciertos y errores para calcular la tasa de fallo
    # datos is numpy array
    # pred is numpy array
    def error(self, datos, pred):
        # Aqui se compara la prediccion (pred) con las clases reales y se calcula el error
        hit_rate = sum(datos == pred)/len(pred)
        return 1-hit_rate # miss rate is the complement of hit rate


    # Realiza una clasificacion utilizando una estrategia de particionado determinada
    # TODO: implementar esta funcion
    def validacion(self, particionado, dataset, clasificador, seed=None):
        # Creamos las particiones siguiendo la estrategia llamando a particionado.creaParticiones
        # - Para validacion cruzada: en el bucle hasta nv entrenamos el clasificador con la particion de train i
        # y obtenemos el error en la particion de test i
        # - Para validacion simple (hold-out): entrenamos el clasificador con la particion de train
        # y obtenemos el error en la particion test. Otra opci�n es repetir la validaci�n simple un n�mero especificado de veces, obteniendo en cada una un error. Finalmente se calcular�a la media.
        random.seed(seed)
        np.random.shuffle(dataset.datos)
        
        particionado.creaParticiones(dataset.datos, seed)
        
        errores = []

        for particion in particionado.particiones:

            datostrain = dataset.datos[particion.indicesTrain, :]
            datostest = dataset.datos[particion.indicesTest, :]
            
            clasificador.entrenamiento(datostrain, dataset.nominalAtributos, dataset.diccionario)
            
            pred = clasificador.clasifica(datostest, dataset.nominalAtributos, dataset.diccionario)
            
            ydatos = datostest[:,-1]            
            err = clasificador.error(ydatos, pred)

            errores.append(err)
        
        return np.mean(errores)
        
        

##############################################################################

class ClasificadorNaiveBayes(Clasificador):
    
    def __init__(self):
        self.prior_probs = []
        self.likelihoods = []

    
    def _multinomialNB(self, xdata, ydata, feat_idx, diccionario):
        n_xi = len(diccionario[feat_idx])
        n_classes = len(diccionario[-1])
        theta_mtx = np.zeros((n_xi, n_classes))
        
        for value in diccionario[feat_idx]:
            feat_val_idx = diccionario[feat_idx][value]
            for class_name in diccionario[-1]:
                class_idx = diccionario[-1][class_name]
                theta_mtx[feat_val_idx][class_idx] = sum((xdata[:,feat_idx] == feat_val_idx)&(ydata == class_idx))/sum(ydata == class_idx)
        
        return theta_mtx
                      

    def _gaussianNB(self, xdata, ydata, feat_idx, diccionario):
        n_classes = len(diccionario[-1])
        
        theta_mtx = np.zeros((n_classes, 2)) # 2 columns: mean and variance for each class
        
        for class_name in diccionario[-1]:
            class_idx = diccionario[-1][class_name]
            # We calculate the mean coditioned to each possible class
            mean_sum = sum(elem for (idx, elem) in enumerate(xdata[:,feat_idx]) if ydata[idx]==class_idx)
            mean_class = mean_sum/sum(ydata == class_idx)
            # We calculate the variance conditioned to each possible class
            var_sum = sum((elem-mean_class)**2 for (idx, elem) in enumerate(xdata[:,feat_idx]) if ydata[idx]==class_name)
            var_class = var_sum/sum(ydata == class_idx)

            theta_mtx[class_idx][0] = mean_class
            theta_mtx[class_idx][1] = var_class
        
        return theta_mtx
            


    def entrenamiento(self,datostrain,atributosDiscretos,diccionario):
        xdata = datostrain[:,:-1] # all rows, all columns but last one
        ydata = datostrain[:,-1]  # all rows, just last column
        
        m, n = xdata.shape     # number of examples, number of features
        n_classes = len(diccionario[-1])  # number of different classes
        
        # Calculating prior probabilities
        self.prior_probs = np.zeros(n_classes) # initializing array of prior probs with zeros
        for class_name in diccionario[-1]:
            class_idx = diccionario[-1][class_name]
            self.prior_probs[class_idx] = sum((class_idx == ydata))/m # P(y=i) = count(ydata==i)/len(ydata)
        
        likelihoods_list = []
        # Calculating likelihoods
        for feat_idx in range(n):
            if atributosDiscretos[feat_idx]:
                # calculating frequentist probs for discrete features
                theta_mtx = self._multinomialNB(xdata, ydata, feat_idx, diccionario)
            else:
                # calculating means and variances for continuous features
                theta_mtx = self._gaussianNB(xdata, ydata, feat_idx, diccionario)

            likelihoods_list.append(theta_mtx)
        
        self.likelihoods = np.asarray(likelihoods_list)

    

    # TODO: implementar
    def clasifica(self,datostest,atributosDiscretos,diccionario):
        xdata = datostest[:,:-1] # all rows, all columns but last one
        ydata = datostest[:,-1]  # all rows, just last column
        
        ndata, n_feat = xdata.shape     # number of examples, number of features
        n_classes = len(diccionario[-1])  # number of different classes
        
        pred = []
        for i in range(ndata):
            classes_probs = []
            for k in range(n_classes):
                class_p = self.prior_probs[k]
                for feat_idx in range(n_feat):
                    if atributosDiscretos[feat_idx]:
                        class_p *= self.likelihoods[feat_idx][int(xdata[i][feat_idx])][k]
                    else:
                        mean = self.likelihoods[feat_idx][k][0]
                        var = self.likelihoods[feat_idx][k][1]
                        class_p *= norm.pdf(xdata[i][feat_idx], loc=mean, scale=math.sqrt(var))
                classes_probs.append(class_p)
            pred.append(classes_probs.index(max(classes_probs)))
            
        return np.asarray(pred)


In [10]:
from sklearn.naive_bayes import MultinomialNB
from Datos import Datos
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import random, math
from scipy.stats import norm

In [11]:
from Datos import Datos
dataset_ttt = Datos("../ConjuntosDatos/tic-tac-toe.data")
ndata, nfeat = dataset_ttt.datos.shape
print(ndata)

958


In [21]:
np.random.shuffle(dataset_ttt.datos)
NB = ClasificadorNaiveBayes()
NB.entrenamiento(dataset_ttt.datos[:int(ndata*(7/8))], dataset_ttt.nominalAtributos, dataset_ttt.diccionario)
pred = NB.clasifica(dataset_ttt.datos[int(ndata*(7/8)):], dataset_ttt.nominalAtributos, dataset_ttt.diccionario)
print(pred)
print("\n")

ydatos = dataset_ttt.datos[int(ndata*(7/8)):,-1]

error = NB.error(ydatos, pred)
print(error)

[1 1 0 1 1 1 0 0 1 1 0 0 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 0 0 1 1 0 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 0 0 1 1 1 1 1 0 0 0 1 1 1
 1 1 1 0 1 1 1 0 1 1 1 0 1 1 1 1 1 1 1 1 1 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0
 1 1 1 1 1 0 1 1 0]


0.2666666666666667


In [22]:
import EstrategiaParticionado

In [23]:
strat_simple = EstrategiaParticionado.ValidacionSimple(0.8)

In [24]:
NB.validacion(strat_simple, dataset_ttt, NB)

0.29166666666666663

In [25]:
strat_simple_rep3 = EstrategiaParticionado.ValidacionSimple(0.8, nreps=3)

In [31]:
NB.validacion(strat_simple_rep3, dataset_ttt, NB)

0.28645833333333337

In [27]:
strat_cross = EstrategiaParticionado.ValidacionCruzada(k_fold=5)

In [28]:
NB.validacion(strat_cross, dataset_ttt, NB)

0.2994764397905759

In [33]:
ttt_db = Datos('../ConjuntosDatos/tic-tac-toe.data')
np.random.shuffle(ttt_db.datos)
X = ttt_db.datos[:,:-1]
y = ttt_db.datos[:,-1]


enc = OneHotEncoder(sparse=False)
X = np.array(enc.fit_transform(X)) 

n, m = X.shape

x_train = X[:int(0.8*n), :]
y_train = y[:int(0.8*n)]

print(x_train.shape, y_train.shape)

x_test = X[int(0.8*n):, :]
y_test = y[int(0.8*n):]

(766, 27) (766,)


/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [34]:
clf = MultinomialNB()

In [35]:
clf.fit(x_train,y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [36]:
clf.score(x_test,y_test)

0.671875

In [37]:
dataset_ger = Datos("../ConjuntosDatos/german.data")
ndata, nfeat = dataset_ger.datos.shape
print(ndata, nfeat)

1000 21


In [38]:
strat_simple_ger = EstrategiaParticionado.ValidacionSimple(0.8)
NB.validacion(strat_simple_ger, dataset_ger, NB)

NameError: name 'idx' is not defined

In [45]:
strat_simple_rep3_ger = EstrategiaParticionado.ValidacionSimple(0.8, nreps=3)
NB.validacion(strat_simple_rep3_ger, dataset_ger, NB)

0.25333333333333335

In [48]:
strat_cross_ger = EstrategiaParticionado.ValidacionCruzada(k_fold=5)
NB.validacion(strat_cross_ger, dataset_ger, NB)

0.25